In [1]:
# coding=utf-8
import math
from scapy.all import *
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd

hosts = dict([('www.easternct.edu','149.152.33.32'), ('web.mit.edu','23.197.229.238')])

DEFAULT_RTT_VALUE = 0

ATTEMPS = 30

def enviar_paquete(host, time_to_live):
    packet = IP(dst=host, ttl=time_to_live) / ICMP()
    res = sr(packet, timeout=1, verbose=0)
    return res[0][ICMP]

In [2]:
def trace_route(hosts):
    
    hops = []
    
    for host in hosts:
        ttl_without_response = []
        ip = hosts[host]
        tipo = 11
        ttl = 1
        while (tipo!=0 and ttl < 30):
            for i in range(0,ATTEMPS):
                if (not ttl in ttl_without_response):
                    ti = time.time()
                    res1 = enviar_paquete(ip, ttl)
                    tf = time.time()
                    rtt = tf-ti
                    if (len(res1) > 0):
                        tipo = res1[0][1].type
                        src = res1[0][1].src
                        if (tipo > 0):
                            hops.append([host,ttl,i,src,rtt])
                        else:
                            hops.append([host,ttl,i,src,rtt])
                    else:
                        hops.append([host,ttl,i,None,DEFAULT_RTT_VALUE])
                        ttl_without_response.append(ttl)
            ttl = ttl + 1

    hops_info = pd.DataFrame(hops,columns=['host','ttl','i','ip','rtt'])
    return hops_info

In [187]:
df = trace_route(hosts)
df_indexed = df.set_index(['host','ttl','i'])
df_indexed

ip       rtt
host              ttl i                           
web.mit.edu       1   0             None  0.000000
                  2   0             None  0.000000
                  3   0             None  0.000000
                  4   0             None  0.000000
                  5   0             None  0.000000
                  6   0   200.89.161.101  0.063950
                      1   200.89.161.101  0.067979
                      2   200.89.161.101  0.068022
                      3   200.89.161.101  0.067797
                      4   200.89.161.101  0.052200
                      5   200.89.161.101  0.083740
                      6   200.89.161.101  0.075924
                      7   200.89.161.101  0.055952
                      8   200.89.161.101  0.155953
                      9   200.89.161.101  0.075816
                      10  200.89.161.101  0.079881
                      11  200.89.161.101  0.067870
                      12  200.89.161.101  0.083911
                      13  200.89.161.101  0.076014
                      14  200.89.161.101  0.087813
                      15  200.89.161.101  0.107785
                      16  200.89.161.101  0.103959
                      17  200.89.161.101  0.087885
                      18  200.89.161.101  0.139920
                      19  200.89.161.101  0.139921
                      20  200.89.161.101  0.095932
                      21  200.89.161.101  0.080024
                      22  200.89.161.101  0.079901
                      23  200.89.161.101  0.076033
                      24  200.89.161.101  0.067719
...                                  ...       ...
www.easternct.edu 16  13   72.10.110.141  0.220069
                      14   72.10.110.141  0.212021
                      15   72.10.110.141  0.207674
                      16   72.10.110.141  0.207940
                      17   72.10.110.141  0.203897
                      18   72.10.110.141  0.211910
                      19   72.10.110.141  0.211964
                      20   72.10.110.141  0.219846
                      21   72.10.110.141  0.203929
                      22   72.10.110.141  0.211910
                      23   72.10.110.141  0.216045
                      24   72.10.110.141  0.211777
                      25   72.10.110.141  0.224052
                      26   72.10.110.141  0.227916
                      27   72.10.110.141  0.223746
                      28   72.10.110.141  0.212105
                      29   72.10.110.141  0.199774
                  17  0             None  0.000000
                  18  0             None  0.000000
                  19  0             None  0.000000
                  20  0             None  0.000000
                  21  0             None  0.000000
                  22  0             None  0.000000
                  23  0             None  0.000000
                  24  0             None  0.000000
                  25  0             None  0.000000
                  26  0             None  0.000000
                  27  0             None  0.000000
                  28  0             None  0.000000
                  29  0             None  0.000000

[277 rows x 2 columns]

In [3]:
import logging
import sys
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)
from scapy.all import *
dst_ip = "web.mit.edu"
src_port = 400
dst_port=80
 
tcp_connect_scan_resp = sr1(IP(dst=dst_ip)/TCP(sport=src_port,dport=dst_port,flags="S"),timeout=10)
if(tcp_connect_scan_resp is None):
    print ("The port is Closed")
    sys.exit()
     
print ("The flags:" + str (tcp_connect_scan_resp.getlayer(TCP).flags))    
    
if(tcp_connect_scan_resp.haslayer(TCP)):
    if(tcp_connect_scan_resp.getlayer(TCP).flags == 0x12):
        #send_rst = sr(IP(dst=dst_ip)/TCP(sport=src_port,dport=dst_port,flags="AR"),timeout=3)
        print ("The port is Open")
        print tcp_connect_scan_resp.getlayer(IP).src
        sys.exit ();
            
#(tcp_connect_scan_resp.getlayer(TCP).flags == 0x14):
print ("The port is Closed ")

Begin emission:
Finished to send 1 packets.

Received 74 packets, got 0 answers, remaining 1 packets
The port is Closed


SystemExit: 

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
df.set_index?


In [4]:
sr1(IP(dst="8.8.8.8")/UDP(dport=53)/DNS(rd=1, qd=DNSQR(qname="www.google.com", qclass="IN", qtype="A")),verbose=0)

<IP  version=4L ihl=5L tos=0x0 len=316 id=45698 flags= frag=0L ttl=55 proto=udp chksum=0xff17 src=8.8.8.8 dst=192.168.0.95 options=[] |<UDP  sport=domain dport=domain len=296 chksum=0x34a6 |<DNS  id=0 qr=1L opcode=QUERY aa=0L tc=0L rd=1L ra=1L z=0L ad=0L cd=0L rcode=ok qdcount=1 ancount=16 nscount=0 arcount=0 qd=<DNSQR  qname='www.google.com.' qtype=A qclass=IN |> an=<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.246' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.217' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.216' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.236' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.212' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.242' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata='181.47.248.226' |<DNSRR  rrname='www.google.com.' type=A rclass=IN ttl=89 rdata=

In [5]:
import dns.resolver
my_resolver = dns.resolver.Resolver()
my_resolver.nameservers = ['8.8.8.8']

answer=my_resolver.query("google.com", "A")
for data in answer:
    print data.address

ImportError: No module named dns.resolver

In [6]:
dns.rdtypes.IN.A.A.address

NameError: name 'dns' is not defined

In [1]:
#!/usr/bin/env python
# coding: utf-8

from __future__ import print_function, unicode_literals, with_statement
import argparse
import contextlib
import requests
import sys
import csv
import matplotlib
# Anti-Grain Geometry (AGG) backend so PyGeoIpMap can be used 'headless'
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pygeoip


def get_ip(ip_file):
    """
    Returns a list of IP addresses from a file containing one IP per line.
    """
    with contextlib.closing(ip_file):
        return [line.strip() for line in ip_file]


def get_lat_lon(ip_list=[], lats=[], lons=[]):
    """
    This function connects to the FreeGeoIP web service to get info from
    a list of IP addresses.
    Returns two lists (latitude and longitude).
    """
    print("Processing {} IPs...".format(len(ip_list)))
    for ip in ip_list:
        r = requests.get("https://freegeoip.net/json/" + ip)
        json_response = r.json()
        print("{ip}, {region_name}, {country_name}, {latitude}, {longitude}".format(**json_response))
        if json_response['latitude'] and json_response['longitude']:
            lats.append(json_response['latitude'])
            lons.append(json_response['longitude'])
    return lats, lons


def geoip_lat_lon(gi, ip_list=[], lats=[], lons=[]):
    """
    This function uses the MaxMind library and databases to geolocate IP addresses
    Returns two lists (latitude and longitude).
    """
    print("Processing {} IPs...".format(len(ip_list)))
    for ip in ip_list:
        try:
            r = gi.record_by_addr(ip)
        except Exception:
            print("Unable to locate IP: %s" % ip)
            continue
        if r is not None:
            print("%s {country_code} {latitude}, {longitude}".format(**r) % ip)
            lats.append(r['latitude'])
            lons.append(r['longitude'])
    return lats, lons


def get_lat_lon_from_csv(csv_file, lats=[], lons=[]):
    """
    Retrieves the last two rows of a CSV formatted file to use as latitude
    and longitude.
    Returns two lists (latitudes and longitudes).
    Example CSV file:
    119.80.39.54, Beijing, China, 39.9289, 116.3883
    101.44.1.135, Shanghai, China, 31.0456, 121.3997
    219.144.17.74, Xian, China, 34.2583, 108.9286
    64.27.26.7, Los Angeles, United States, 34.053, -118.2642
    """
    with contextlib.closing(csv_file):
        reader = csv.reader(csv_file)
        for row in reader:
            lats.append(row[-2])
            lons.append(row[-1])

    return lats, lons


def generate_map(output, lats=[], lons=[], wesn=None):
    """
    Using Basemap and the matplotlib toolkit, this function generates a map and
    puts a red dot at the location of every IP addresses found in the list.
    The map is then saved in the file specified in `output`.
    """
    print("Generating map and saving it to {}".format(output))
    if wesn:
        wesn = [float(i) for i in wesn.split('/')]
        m = Basemap(projection='cyl', resolution='l',
                llcrnrlon=wesn[0], llcrnrlat=wesn[2],
                urcrnrlon=wesn[1], urcrnrlat=wesn[3])
    else:
        m = Basemap(projection='cyl', resolution='l')
    m.bluemarble()
    x, y = m(lons, lats)
    m.scatter(x, y, s=1, color='#ff0000', marker='o', alpha=0.3)
    plt.savefig(output, dpi=300, bbox_inches='tight')







In [30]:
ip_list = ['218.60.148.32', '59.63.175.24', '109.207.56.22', '59.63.175.25', '59.39.71.222', '222.186.62.17', '72.80.16.100', '60.199.196.144']
gi = pygeoip.GeoIP('GeoLiteCity.dat')
lats, lons = geoip_lat_lon(gi, ip_list)
generate_map('salida.png', lats, lons) #, wesn='-12/45/30/65'

Processing 8 IPs...
218.60.148.32 CN 41.7922, 123.4328
59.63.175.24 CN 28.55, 115.9333
109.207.56.22 PL 50.9333, 17.3
59.63.175.25 CN 28.55, 115.9333
59.39.71.222 CN 23.1167, 113.25
222.186.62.17 CN 32.0617, 118.7778
72.80.16.100 US 41.0051, -73.7846
60.199.196.144 TW 25.0418, 121.4966
Generating map and saving it to salida.png
